# Colombiacheck Demo

Colombiacheck es un proyecto periodístico de investigación desarrollado en Colombia, que ayuda a la verificación de información emitida por importantes personajes públicos en redes sociales o portales noticiosos, sobre asuntos
relevantes para el debate público. Su metodología de análisis y clasificación está basada en lo siguiente:

* Realizar una selección de publicaciones relevantes para el debate público que
circula por las redes sociales o por importantes portales noticiosos.
* Realizar una búsqueda de la veracidad de las publicaciones seleccionadas
contrastando hechos, datos y fuentes utilizadas.
* Determinar la clasificación de las publicaciones con base a un conjunto de
calificaciones definidas por Colombiacheck, estas calificaciones son:

  1. **Verdadero:** Datos que corresponden exactamente a la realidad de los hechos.
  2. **Verdadero, pero…:** Están alineados a los datos más recientes, pero dejan por fuera alguna parte importante del contexto.
  3. **Cuestionable:** Están alineados a los datos más recientes, pero se usan para llegar a conclusiones erróneas o son interpretados erróneamente.
  4. **Falso:** Los datos son contrarios a la realidad de los hechos.
  5. **Inchequeable:** Datos que no son confiables, porque vienen de una opinión

In [1]:
import json
import requests
import pandas as pd

from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'requests'

In [ ]:
def dataset(df, text_title, text_paragraph, verdict):
  for i in range((len(text_title))):
    title = str(text_title[i]).split('>')[1].split('<')[0]
    paragraph = str(text_paragraph[i]).split('>')[1].split('<')[0]
    verd = str(verdict[i]).split('>')[1].split('<')[0]
    df.loc[len(df)] = [title, paragraph, verd]
  return df

In [ ]:
url = 'https://colombiacheck.com/chequeos?page='
df = pd.DataFrame(columns=['statement','statement_paragraph','verdict'])

for i in range(200):
  colcheck = requests.get(f'{url + str(i)}')
  if str(colcheck.status_code) == '200':
    s = BeautifulSoup(colcheck.text, 'lxml')
    statement = s.find('div', attrs={'id':'bloqueImagenes'}).find_all('h3', class_='Chequeo-texto-titulo')
    statement_paragraph = s.find('div', attrs={'id':'bloqueImagenes'}).find_all('p', class_='Chequeo-texto-parrafo')
    verdict = s.find('div', attrs={'id':'bloqueImagenes'}).find_all('p', class_='Chequeo-picture-bandera-text')

    df = dataset(df, text_title, text_paragraph, verdict)
  else:
    print(f'Status: {colcheck.status_code}')


In [ ]:
df.head()

,text_title,text_paragraph,verdict
0,\n Video de Petro borracho en el que ha...,"\n La alteración, que circula desde la ...",Falso
1,\n Circulan de nuevo trinos falsos de p...,\n Desde 2019 hemos desmentido el monta...,Falso
2,\n De nuevo: Supuesta entrevista de El ...,\n Esta es la segunda vez que chequeamo...,Falso
3,\n ONU no investigará por narcotráfico ...,\n Con la intervención de un vocero de ...,Falso
4,\n No se puede asegurar que renuncia de...,\n La aplicación y los detectores de in...,Inchequeable


In [ ]:
df.shape

(1600, 3)

In [ ]:
df = df.replace('\n','', regex=True)

In [ ]:
df.head()

,text_title,text_paragraph,verdict
0,Video de Petro borracho en el que habl...,"La alteración, que circula desde la ca...",Falso
1,Circulan de nuevo trinos falsos de pel...,Desde 2019 hemos desmentido el montaje...,Falso
2,De nuevo: Supuesta entrevista de El Ti...,Esta es la segunda vez que chequeamos ...,Falso
3,ONU no investigará por narcotráfico y ...,Con la intervención de un vocero de un...,Falso
4,No se puede asegurar que renuncia de I...,La aplicación y los detectores de inte...,Inchequeable


In [ ]:
df.to_csv('/content/Colombiancheck.csv')

In [ ]:
def clean_text(statement, statement_paragraph, verdict, links):
  data = []
  # print(f'{len(links)}  {len(statement)}')
  for i in range(len(statement)):
    s = str(statement[i]).split('>')[1].split('<')[0].replace('\n','')
    sp = str(statement_paragraph[i]).split('>')[1].split('<')[0].replace('\n','')
    v = str(verdict[i]).split('>')[1].split('<')[0].replace('\n','')

    factcheck_analysis_link = ''
    try:
      factcheck_analysis_link = links[i]
    except IndexError:
      factcheck_analysis_link = 'none'

    data.append({"statement":s,
                 "statement_paragraph":sp,
                 "verdict":v,
                 "factcheck_analysis_link":factcheck_analysis_link})
  return data

In [ ]:
url = 'https://colombiacheck.com/chequeos?page='
data = []
for i in range(500):
  colcheck = requests.get(f'{url + str(i)}')
  if str(colcheck.status_code) == '200':
    s = BeautifulSoup(colcheck.text, 'lxml')
    statement = s.find('div', attrs={'id':'bloqueImagenes'}).find_all('h3', class_='Chequeo-texto-titulo')
    statement_paragraph = s.find('div', attrs={'id':'bloqueImagenes'}).find_all('p', class_='Chequeo-texto-parrafo')
    verdict = s.find('div', attrs={'id':'bloqueImagenes'}).find_all('p', class_='Chequeo-picture-bandera-text')
    links = []
    for link in s.find('div', attrs={'id':'bloqueImagenes'}).find_all('a'):
      if 'chequeos' in link['href']:
          links.append(link['href'])

    factcheck_analysis_link = f'{url + str(i)}'
    d = clean_text(statement, statement_paragraph, verdict, links)

    data.append(d)
  else:
    print(f'Status: {colcheck.status_code}')

In [ ]:
with open('/content/ColombianCheck.json', 'w') as f:
    json.dump(data, f)

In [ ]:
data

[[{'statement': '        La mayoría de hallazgos de Contraloría por cerca de $70.000 millones en MinMinas son anteriores a Irene Vélez \t\t\t',
   'statement_paragraph': '        El informe usado para atacar a la saliente ministra en realidad se refiere a 2021 y 2022, por lo que cubre solo cinco meses de su gestión y 19 del gobierno anterior.      ',
   'verdict': 'Cuestionable',
   'factcheck_analysis_link': '/chequeos/la-mayoria-de-hallazgos-de-contraloria-por-cerca-de-70000-millones-en-minminas-es-anterior'},
  {'statement': '        Video de Petro borracho en el que habla del ‘hígado de la Colombia Humana’ es un montaje\t\t\t',
   'statement_paragraph': '        La alteración, que circula desde la campaña presidencial de 2022, se nota porque la voz no va acorde con el movimiento de los labios, entre otros detalles.      ',
   'verdict': 'Falso',
   'factcheck_analysis_link': '/chequeos/video-de-petro-borracho-en-el-que-habla-del-higado-de-la-colombia-humana-es-un-montaje'},
  {'sta